In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import math
from scipy.spatial import distance
import random 
from statistics import mean
from scipy.stats import multivariate_normal
from scipy.stats import norm

### Task 1

#### 1i

In [ ]:
col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Class']
iris_data = pd.read_csv('UCIdata-exercise1/iris.data', names = col_names)

def knn(data, query, k) : #data will be in the form of a DataFrame with the first columns been used to calculate the nearest 
    #neighbor and the last one will be used for the classification of each element
    neighbor_distances_and_indices = []
    # 1. For each example in the data
    for index in range(len(data)):
        # 1.1 Calculate the distance between the query example and the current
        # example from the data.
        A=data.loc[index].tolist()[:-1] #transform to list the values of the dimensions of every instance of the DataFrame
        distance1 = distance.euclidean(A, query)
        
        
        # 1.2 Add the distance and the index of the example to an ordered collection
        neighbor_distances_and_indices.append((distance1, index))
    
    # 2. Sort the ordered collection of distances and indices from
    # smallest to largest (in ascending order) by the distances
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    
    # 3. Pick the first K entries from the sorted collection
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k]
    
    # 4. Get the labels of the selected K entries
    k_nearest_labels = [data.loc[i][-1] for distance, i in k_nearest_distances_and_indices]

    return Counter(k_nearest_labels).most_common(1)[0][0] #, k_nearest_distances_and_indices 

def partition (df, n): # in order to perform k - fold validation we split the dataframe into n - random dataframes 
#and create a list containing them
    df=df.sample(frac = 1)
    cols=df.columns.tolist()
    split_df=[]
    n1=0
    n2=len(df)//n
    if len(df)//n==0:
        for i in range(n):
            split_df.append(df.iloc[n1:n2])
            n1=n2
            n2=n2+len(df)//n
        return(split_df)
    else:
        residual=len(df)%n
        for i in range(n):
            if n-i-1>residual:
                split_df.append(df.iloc[n1:n2])
                n1=n2
                n2=n2+len(df)//n
            else:
                split_df.append(df.iloc[n1:n2])
                n1=n2
                n2=n2+len(df)//n+1
        return(split_df)
            

#We will implement 5 -fold, 10 -fold and 15 -fold validation to the iris dataset. The k for each time implementing the knn will 
#be an even number each time, since there are 3 different target Classes (odd number of classes). So k will be 2,4,6 or 8
#each time

for n_fold in ([5,10,15]):
    splited=partition(iris_data,n_fold)
    for k in range(2,9,2):
        accuracy_list=[]
        for i in range(len(splited)):
            splited_2=splited[:]
            test_data=splited_2.pop(i)
            train_data=pd.concat(splited_2)
            train_data.index=range(len(train_data))
            counter_accuracy=0
            for j in range(len(test_data)):
                if knn(train_data,test_data.iloc[j][:4].tolist(),k)==test_data.iloc[j][4]:
                    counter_accuracy+=1
            accuracy=counter_accuracy/len(test_data)
            accuracy_list.append(accuracy)
        final_accuracy=mean(accuracy_list)
        print ('Accuracy of implementing the ',k,'-NN algorithm and a ',n_fold,'-fold cross validation is: ',100*final_accuracy,'%')



 

Accuracy of implementing the  2 -NN algorithm and a  5 -fold cross validation is:  96.0 %
Accuracy of implementing the  4 -NN algorithm and a  5 -fold cross validation is:  96.0 %
Accuracy of implementing the  6 -NN algorithm and a  5 -fold cross validation is:  95.33333333333334 %
Accuracy of implementing the  8 -NN algorithm and a  5 -fold cross validation is:  96.0 %
Accuracy of implementing the  2 -NN algorithm and a  10 -fold cross validation is:  96.0 %
Accuracy of implementing the  4 -NN algorithm and a  10 -fold cross validation is:  96.0 %
Accuracy of implementing the  6 -NN algorithm and a  10 -fold cross validation is:  96.0 %
Accuracy of implementing the  8 -NN algorithm and a  10 -fold cross validation is:  96.0 %
Accuracy of implementing the  2 -NN algorithm and a  15 -fold cross validation is:  96.0 %
Accuracy of implementing the  4 -NN algorithm and a  15 -fold cross validation is:  96.0 %
Accuracy of implementing the  6 -NN algorithm and a  15 -fold cross validation is

#### 1ii

So the best Algorithm to be implemented in the iris dataset is the 8 NN, since it has averaged a 96,44 % accuracy in 10-fold, 20- fold and 30-fold cross valdiation respectively

In [ ]:
col_names = ['Pregnancies','Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age','Class']
pima_data = pd.read_csv('UCIdata-exercise1/pima-indians-diabetes.data', names = col_names)

# There are values in  columns that represent health tests  that should have value >0 such as Insulin.
#These values will be replaced by the average of the column they are included
for col in ['Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age']:
    val = pima_data[col].mean()
    pima_data[col] = pima_data[col].replace(0, val)
    
#We will implement  10 -fold, 20 -fold and 30-fold validation to the pima dataset. The k for each time implementing the knn 
#will be an odd number each time, since there are 2 different target Classes (even number of classes). 
#So k in that case will be 5,9,13,17 each time


#Since the dataset is larger that the previous one (iris), we will create another function that will accept as input a dataframe
# and a list with values of the dimensions represented in that dataframe. As a result, it will calculate all the euclidean 
#distances between all the elements of the dataframe and the list, and it will reuturn a list of all the sorted distances, 
#and at what row of the dataframe each distance occurs. In that way,for each N-fold validation, we will get a sorted number of 
#the distances, and later we will extract the first 5,9,13 or 17 elements, depending on whether we are implementing the 
#5-NN, 9-NN,13-NN or 13-NN algorithm


def euclidean_1(data, query) : #data will be in the form of a DataFrame with the first columns been used to calculate the
    #nearest  neighbor and the last one will be used for the classification of each element
   
    # it returns the sorted euclidean distance as well as the index of the row of the dataframe data in which the smaller 
    #euclidean distance to the list "query" ,which represents a list of dimenions' values of a datapoint , occurs
    
    neighbor_distances_and_indices = []
    # 1. For each example in the data
    for index in range(len(data)):
        # 1.1 Calculate the distance between the query example and the current
        # example from the data.
        A=data.loc[index].tolist()[:-1] #transform to list the values of the dimensions of every instance of the DataFrame
        distance1 = distance.euclidean(A, query)
        
        
        # 1.2 Add the distance and the index of the example to an ordered collection
        neighbor_distances_and_indices.append((distance1, index))
    
    # 2. Sort the ordered collection of distances and indices from
    # smallest to largest (in ascending order) by the distances
    return (sorted(neighbor_distances_and_indices))
    

for n_fold in ([10,20,30]): #10-fold, 20-fold or 30-fold cross validation
    splited=partition(pima_data,n_fold)
    
    accuracy_list_5=[] #this list will be filled with the accuracies of the 5-NN algorithm, for each of the 
    # N-fold cross validations each time
    accuracy_list_9=[] #the same as above
    accuracy_list_13=[]
    accuracy_list_17=[]
    for i in range(len(splited)):
        splited_2=splited[:]
        test_data=splited_2.pop(i)
        train_data=pd.concat(splited_2)
        train_data.index=range(len(train_data))
        counter_accuracy_5=0
        counter_accuracy_9=0
        counter_accuracy_13=0
        counter_accuracy_17=0
        for j in range(len(test_data)):
            distance2=euclidean_1(train_data,test_data.iloc[j][:8].tolist())
            #5-nn calculations
            _5_nearest_distances_and_indices = distance2[:5]
            _5_nearest_labels = [train_data.loc[i][-1] for distance, i in _5_nearest_distances_and_indices]
            if Counter(_5_nearest_labels).most_common(1)[0][0]==test_data.iloc[j][8]:
                counter_accuracy_5+=1
                
            #9-nn
            _9_nearest_distances_and_indices = distance2[:9]
            _9_nearest_labels = [train_data.loc[i][-1] for distance, i in _9_nearest_distances_and_indices]
            if Counter(_9_nearest_labels).most_common(1)[0][0]==test_data.iloc[j][8]:
                counter_accuracy_9+=1
                
            #13-nn
            _13_nearest_distances_and_indices = distance2[:13]
            _13_nearest_labels = [train_data.loc[i][-1] for distance, i in _13_nearest_distances_and_indices]
            if Counter(_13_nearest_labels).most_common(1)[0][0]==test_data.iloc[j][8]:
                counter_accuracy_13+=1
                
            #17-nn
            _17_nearest_distances_and_indices = distance2[:17]
            _17_nearest_labels = [train_data.loc[i][-1] for distance, i in _17_nearest_distances_and_indices]
            if Counter(_17_nearest_labels).most_common(1)[0][0]==test_data.iloc[j][8]:
                counter_accuracy_17+=1
                
        accuracy_list_5.append(counter_accuracy_5/len(test_data))
        accuracy_list_9.append(counter_accuracy_9/len(test_data))
        accuracy_list_13.append(counter_accuracy_13/len(test_data))
        accuracy_list_17.append(counter_accuracy_17/len(test_data))
    print ('Accuracy of implementing the ',5,'-NN algorithm and a ',n_fold,'-fold cross validation is: ',100*mean(accuracy_list_5),'%')    
    print ('Accuracy of implementing the ',9,'-NN algorithm and a ',n_fold,'-fold cross validation is: ',100*mean(accuracy_list_9),'%')    
    print ('Accuracy of implementing the ',13,'-NN algorithm and a ',n_fold,'-fold cross validation is: ',100*mean(accuracy_list_13),'%')    
    print ('Accuracy of implementing the ',17,'-NN algorithm and a ',n_fold,'-fold cross validation is: ',100*mean(accuracy_list_17),'%')    
    
    


Accuracy of implementing the  5 -NN algorithm and a  10 -fold cross validation is:  72.00615174299385 %
Accuracy of implementing the  9 -NN algorithm and a  10 -fold cross validation is:  73.95591250854409 %
Accuracy of implementing the  13 -NN algorithm and a  10 -fold cross validation is:  73.69104579630896 %
Accuracy of implementing the  17 -NN algorithm and a  10 -fold cross validation is:  73.69958988380041 %
Accuracy of implementing the  5 -NN algorithm and a  20 -fold cross validation is:  70.68825910931174 %
Accuracy of implementing the  9 -NN algorithm and a  20 -fold cross validation is:  73.5425101214575 %
Accuracy of implementing the  13 -NN algorithm and a  20 -fold cross validation is:  73.5391363022942 %
Accuracy of implementing the  17 -NN algorithm and a  20 -fold cross validation is:  73.27935222672065 %
Accuracy of implementing the  5 -NN algorithm and a  30 -fold cross validation is:  71.9025641025641 %
Accuracy of implementing the  9 -NN algorithm and a  30 -fold c

So the best Algorithm to be implemented in the pima dataset is the 17 NN, since it has averaged a 73.82 % accuracy in 10-fold, 20- fold and 30-fold cross valdiation respectively

### Task 2

#### 2a

In [ ]:
# Firstly for the iris data
col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Class']
iris_data = pd.read_csv('UCIdata-exercise1/iris.data', names = col_names)

#firstly we will seperate 3 classes and its respective features. We will create 3 different arrays. Each element of every array
#will include 4 dimensions
setosa=iris_data[iris_data["Class"] == 'Iris-setosa'].drop(["Class"], axis=1).to_numpy()
versicolor=iris_data[iris_data["Class"] == 'Iris-versicolor'].drop(["Class"], axis=1).to_numpy()
virginica=iris_data[iris_data["Class"] == 'Iris-virginica'].drop(["Class"], axis=1).to_numpy()


#Since for every class the Covariance matrices are diagonal, with all diagonal elements equal  one can assume to have 
#only a diagonal covariance matrix and one can estimate 
#the mean and the variance in each dimension separately and describe the multivariate density function in terms of a 
#product of univariate Gaussians. Moreover, since it is impossible for all 4 variables to have the same variance for each case
#each element of the diagonal covariance matrix will be the the maximum of the four variances (worst case scenario)


# setosa Class
Φ=setosa
mean11=np.sum(Φ,axis=0)/setosa.shape[0]
var=0
for column in range(setosa.shape[1]):
    if np.var(setosa[:,column])>var:
        var=np.var(setosa[:,column])
Σ11=np.identity(setosa.shape[1])*var    #calculate the covariaance matrix
def setosa_pdf_1 (x): #x be a 4-dimensional vector
    pdf= multivariate_normal(mean11,Σ11).pdf(x)
    return pdf

# versicolor Class
Φ=versicolor
mean21=np.sum(Φ,axis=0)/versicolor.shape[0] 
var=0
for column in range(versicolor.shape[1]):
    if np.var(versicolor[:,column])>var:
        var=np.var(versicolor[:,column])
Σ21=np.identity(versicolor.shape[1])*var    #calculate the covariaance matrix
def versicolor_pdf_1 (x): #x be a 4-dimensional vector
    pdf= multivariate_normal(mean21,Σ21).pdf(x)
    return pdf

# virginica Class
Φ=virginica
mean31=np.sum(Φ,axis=0)/virginica.shape[0] 
var=0
for column in range(virginica.shape[1]):
    if np.var(virginica[:,column])>var:
        var=np.var(virginica[:,column])
Σ31=np.identity(virginica.shape[1])*var    #calculate the covariaance matrix
def virginica_pdf_1 (x): #x be a 4-dimensional vector
    pdf= multivariate_normal(mean31,Σ31).pdf(x)
    return pdf

In [ ]:
# Then for the pima data
col_names = ['Pregnancies','Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age','Class']
pima_data = pd.read_csv('UCIdata-exercise1/pima-indians-diabetes.data', names = col_names)
for col in ['Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age']:
    val = pima_data[col].mean()
    pima_data[col] = pima_data[col].replace(0, val)

#firstly we will seperate 2 classes and its respective features. We will create 2 different arrays. Each element of every array
#will include 8 dimensions
yes_diabetes=pima_data[pima_data["Class"] == 1].drop(["Class"], axis=1).to_numpy()
no_diabetes=pima_data[pima_data["Class"] == 0].drop(["Class"], axis=1).to_numpy()

#yes_diabetes
Φ=yes_diabetes
mean41=np.sum(Φ,axis=0)/yes_diabetes.shape[0] #case I.I.D dimensions #case I.I.D dimensions
var=0
for column in range(yes_diabetes.shape[1]):
    if np.var(yes_diabetes[:,column])>var:
        var=np.var(yes_diabetes[:,column])
Σ41=np.identity(yes_diabetes.shape[1])*var    #calculate the covariaance matrix
def yes_diabetes_pdf_1 (x): #x be a 8-dimensional vector
    pdf= multivariate_normal(mean41,Σ41).pdf(x)
    return pdf

#no_diabetes
Φ=no_diabetes
mean51=np.sum(Φ,axis=0)/no_diabetes.shape[0] 
var=0
for column in range(no_diabetes.shape[1]):
    if np.var(no_diabetes[:,column])>var:
        var=np.var(no_diabetes[:,column])
Σ51=np.identity(no_diabetes.shape[1])*var    #calculate the covariaance matrix
def no_diabetes_pdf_1 (x): #x be a 8-dimensional vector
    pdf= multivariate_normal(mean51,Σ51).pdf(x)
    return pdf


#### 2b

In [ ]:
# For the purpose of solution of 2)b we consider the variables to be idependant and identically distributed
# Firstly for the iris data 
col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Class']
iris_data = pd.read_csv('UCIdata-exercise1/iris.data', names = col_names)

#firstly we will seperate 3 classes and its respective features. We will create 3 different arrays. Each element of every array
#will include 4 dimensions
setosa=iris_data[iris_data["Class"] == 'Iris-setosa'].drop(["Class"], axis=1).to_numpy()
versicolor=iris_data[iris_data["Class"] == 'Iris-versicolor'].drop(["Class"], axis=1).to_numpy()
virginica=iris_data[iris_data["Class"] == 'Iris-virginica'].drop(["Class"], axis=1).to_numpy()

# setosa
Φ=setosa
mean12=np.sum(Φ,axis=0)/setosa.shape[0] #case I.I.D dimensions
sum_=0
for i in range(setosa.shape[0]):
    sum_+=np.matmul((Φ[i]-mean12).reshape(4,1),np.transpose((Φ[i]-mean12).reshape(4,1)))
Σ12=sum_/setosa.shape[0]
def setosa_pdf_2 (x): #x be a 4-dimensional vector
    pdf= math.exp((-1/2)*np.matmul(np.matmul(np.transpose((x-mean12).reshape(4,1)),np.linalg.inv(Σ12)),(x-mean12).reshape(4,1)))/(((2*math.pi)**(setosa.shape[1]/2))*(np.linalg.det(Σ12))**(1/2))
    return pdf

# versicolor
Φ=versicolor
mean22=np.sum(Φ,axis=0)/versicolor.shape[0] #case I.I.D dimensions
sum_=0
for i in range(versicolor.shape[0]):
    sum_+=np.matmul((Φ[i]-mean22).reshape(4,1),np.transpose((Φ[i]-mean22).reshape(4,1)))
Σ22=sum_/versicolor.shape[0]
def versicolor_pdf_2 (x): #x be a 4-dimensional vector
    pdf= math.exp((-1/2)*np.matmul(np.matmul(np.transpose((x-mean22).reshape(4,1)),np.linalg.inv(Σ22)),(x-mean22).reshape(4,1)))/(((2*math.pi)**(versicolor.shape[1]/2))*(np.linalg.det(Σ22))**(1/2))
    return pdf

# virginica
Φ=virginica
mean32=np.sum(Φ,axis=0)/virginica.shape[0] #case I.I.D dimensions
sum_=0
for i in range(virginica.shape[0]):
    sum_+=np.matmul((Φ[i]-mean32).reshape(4,1),np.transpose((Φ[i]-mean32).reshape(4,1)))
Σ32=sum_/virginica.shape[0]
def virginica_pdf_2 (x): #x be a 4-dimensional vector
    pdf= math.exp((-1/2)*np.matmul(np.matmul(np.transpose((x-mean32).reshape(4,1)),np.linalg.inv(Σ32)),(x-mean32).reshape(4,1)))/(((2*math.pi)**(virginica.shape[1]/2))*(np.linalg.det(Σ32))**(1/2))
    return pdf


In [ ]:
# For the purpose of solution of 2)b we consider the variables to be idependant and identically distributed
# Then for the pima data
col_names = ['Pregnancies','Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age','Class']
pima_data = pd.read_csv('UCIdata-exercise1/pima-indians-diabetes.data', names = col_names)
for col in ['Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age']:
    val = pima_data[col].mean()
    pima_data[col] = pima_data[col].replace(0, val)

#firstly we will seperate 2 classes and its respective features. We will create 2 different arrays. Each element of every array
#will include 8 dimensions
yes_diabetes=pima_data[pima_data["Class"] == 1].drop(["Class"], axis=1).to_numpy()
no_diabetes=pima_data[pima_data["Class"] == 0].drop(["Class"], axis=1).to_numpy()

#yes_diabetes
Φ=yes_diabetes
mean42=np.sum(Φ,axis=0)/yes_diabetes.shape[0] #case I.I.D dimensions
sum_=0
for i in range(yes_diabetes.shape[0]):
    sum_+=np.matmul((Φ[i]-mean42).reshape(8,1),np.transpose((Φ[i]-mean42).reshape(8,1)))
Σ42=sum_/yes_diabetes.shape[0]
def yes_diabetes_pdf_2 (x): #x be a 8-dimensional vector
    pdf= math.exp((-1/2)*np.matmul(np.matmul(np.transpose((x-mean42).reshape(8,1)),np.linalg.inv(Σ42)),(x-mean42).reshape(8,1)))/(((2*math.pi)**(yes_diabetes.shape[1]/2))*(np.linalg.det(Σ42))**(1/2))
    return pdf

#no_diabetes
Φ=no_diabetes
mean52=np.sum(Φ,axis=0)/no_diabetes.shape[0] #case I.I.D dimensions
sum_=0
for i in range(no_diabetes.shape[0]):
    sum_+=np.matmul((Φ[i]-mean52).reshape(8,1),np.transpose((Φ[i]-mean52).reshape(8,1)))
Σ52=sum_/no_diabetes.shape[0]
def no_diabetes_pdf_2 (x): #x be a 8-dimensional vector
    pdf= math.exp((-1/2)*np.matmul(np.matmul(np.transpose((x-mean52).reshape(8,1)),np.linalg.inv(Σ52)),(x-mean52).reshape(8,1)))/(((2*math.pi)**(no_diabetes.shape[1]/2))*(np.linalg.det(Σ52))**(1/2))
    return pdf



#### 2c

In [ ]:
# PDF for the Naive Bayes Approach
def pdf_naive(x,mean,variance): #mean and variance list of the means and variances of the same length- same as the no of the 
#variables , x an 1-d array of the values of the variables of the same length of the above lists
    pdf=1
    for count in range(len(mean)):
        pdf=(pdf*math.exp((-1/2)*(1/variance[count])*(x[count]-mean[count])**2))/((2*math.pi)**(1/2)*((variance[count])**(1/2)))
    return pdf


In [ ]:
# Firstly for the iris data 

col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Class']
iris_data = pd.read_csv('UCIdata-exercise1/iris.data', names = col_names)

#firstly we will seperate 3 classes and its respective features. We will create 3 different arrays. Each element of every array
#will include 4 dimensions
setosa=iris_data[iris_data["Class"] == 'Iris-setosa'].drop(["Class"], axis=1).to_numpy()
versicolor=iris_data[iris_data["Class"] == 'Iris-versicolor'].drop(["Class"], axis=1).to_numpy()
virginica=iris_data[iris_data["Class"] == 'Iris-virginica'].drop(["Class"], axis=1).to_numpy()

# setosa
Φ=setosa
mean13=(np.sum(Φ,axis=0)/setosa.shape[0]).tolist() #case I.I.D dimensions
Σ13=[]
for column in range(setosa.shape[1]):
    Σ13.append(np.var(setosa[:,column]))

def setosa_pdf_3 (x): #x be a 4-dimensional vector
    pdf= pdf_naive(x,mean13,Σ13)
    return pdf

#versicolor
Φ=versicolor
mean23=(np.sum(Φ,axis=0)/versicolor.shape[0]).tolist() #case I.I.D dimensions
Σ23=[]
for column in range(versicolor.shape[1]):
    Σ23.append(np.var(versicolor[:,column]))

def versicolor_pdf_3 (x): #x be a 4-dimensional vector
    pdf= pdf_naive(x,mean23,Σ23)
    return pdf

#virginica
Φ=virginica
mean33=(np.sum(Φ,axis=0)/virginica.shape[0]).tolist() #case I.I.D dimensions
Σ33=[]
for column in range(virginica.shape[1]):
    Σ33.append(np.var(virginica[:,column]))

def virginica_pdf_3 (x): #x be a 4-dimensional vector
    pdf= pdf_naive(x,mean33,Σ33)
    return pdf




In [ ]:
# Then for the pima data
col_names = ['Pregnancies','Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age','Class']
pima_data = pd.read_csv('UCIdata-exercise1/pima-indians-diabetes.data', names = col_names)
for col in ['Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age']:
    val = pima_data[col].mean()
    pima_data[col] = pima_data[col].replace(0, val)

#firstly we will seperate 2 classes and its respective features. We will create 2 different arrays. Each element of every array
#will include 8 dimensions
yes_diabetes=pima_data[pima_data["Class"] == 1].drop(["Class"], axis=1).to_numpy()
no_diabetes=pima_data[pima_data["Class"] == 0].drop(["Class"], axis=1).to_numpy()

#yes_diabetes
Φ=yes_diabetes
mean43=(np.sum(Φ,axis=0)/yes_diabetes.shape[0]).tolist() #case I.I.D dimensions
Σ43=[]
for column in range(yes_diabetes.shape[1]):
    Σ43.append(np.var(yes_diabetes[:,column]))

def yes_diabetes_pdf_3 (x): #x be a 4-dimensional vector
    pdf= pdf_naive(x,mean43,Σ43)
    return pdf

#no_diabetes
Φ=no_diabetes
mean53=(np.sum(Φ,axis=0)/no_diabetes.shape[0]).tolist() #case I.I.D dimensions
Σ53=[]
for column in range(no_diabetes.shape[1]):
    Σ53.append(np.var(no_diabetes[:,column]))

def no_diabetes_pdf_3 (x): #x be a 4-dimensional vector
    pdf= pdf_naive(x,mean53,Σ53)
    return pdf

#### 2d

In [ ]:
# We will create a function that returns PDF using the Naive Bayes approach,when marginal pdfs are computed using 
#1-d Parzen windows with gaussian kernels 
#Since marginal pdfs are computed using 1-d Parzen windows with gaussian kernels, for the coputation of the PDF of each
#dimension We will use for each dimension the average of n Gaussian functions with each data point as a center. σ needs
#to be predetermined and will be the width h of each parzen window. Then, since naive bayes approach is used, the pdf of 
#each dimension will be multiplied in order to get the final pdf 


def kde_parzen_pdf_naive_bayes(x_kde,x_vector,bw): #x_kde an array with columns as dimensions whose pdfs should be calculated.
    #Note that this function will calculate the pdf of each array seperately using the parzen - windows method with h=bw,
    # and then it will calculate the pdf of the whole array 1-d vector with dimensions as many as the columns of the array x_kde
    #using the naive bayes approach
    pdf=1
    for dimension in range(x_kde.shape[1]):
        sum11=0
        for sample in range (x_kde.shape[0]):
            sum11+=norm(x_kde[sample,dimension],bw).pdf(x_vector[dimension])
        pdf=pdf*sum11/x_kde.shape[0]        
    return pdf




In [ ]:
# Firstly for the iris data 
col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Class']
iris_data = pd.read_csv('UCIdata-exercise1/iris.data', names = col_names)

#firstly we will seperate 3 classes and its respective features. We will create 3 different arrays. Each element of every array
#will include 4 dimensions
setosa=iris_data[iris_data["Class"] == 'Iris-setosa'].drop(["Class"], axis=1).to_numpy()
versicolor=iris_data[iris_data["Class"] == 'Iris-versicolor'].drop(["Class"], axis=1).to_numpy()
virginica=iris_data[iris_data["Class"] == 'Iris-virginica'].drop(["Class"], axis=1).to_numpy()

#h will be the same for all four dimensions of all three differenct classes. We have three different patterns 
#observed in the iris data (setosa, versicolor and virginica) so the h will be the squareroot of 3
h=3**(1/2)

def setosa_pdf_4(x):
    return kde_parzen_pdf_naive_bayes(setosa,x,h)
    
def versicolor_pdf_4(x):
    return kde_parzen_pdf_naive_bayes(versicolor,x,h)

def virginica_pdf_4(x):
    return kde_parzen_pdf_naive_bayes(virginica,x,h)

In [ ]:
# Then for the pima data
col_names = ['Pregnancies','Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age','Class']
pima_data = pd.read_csv('UCIdata-exercise1/pima-indians-diabetes.data', names = col_names)
for col in ['Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age']:
    val = pima_data[col].mean()
    pima_data[col] = pima_data[col].replace(0, val)

#firstly we will seperate 2 classes and its respective features. We will create 2 different arrays. Each element of every array
#will include 8 dimensions
yes_diabetes=pima_data[pima_data["Class"] == 1].drop(["Class"], axis=1).to_numpy()
no_diabetes=pima_data[pima_data["Class"] == 0].drop(["Class"], axis=1).to_numpy()

#h will be the same for all 8 dimensions of all 2 differenct classes. We have 2 different patterns 
#observed in the pima data (1,0) so the h will be the squareroot of 2
h=2**(1/2)

#yes_diabetes
def yes_diabetes_pdf_4(x):
    return kde_parzen_pdf_naive_bayes(yes_diabetes,x,h)

#no_diabetes
def no_diabetes_pdf_4(x):
    return kde_parzen_pdf_naive_bayes(no_diabetes,x,h)

 AIC and BIC penalizes the usage of many parameters for possible overfitting. So lets calculate the parameters used for every model  and the log likelihood for all models and all classes. Remember that the best model for each class is the one that has achieved the lowest score

In [ ]:
# Firstly for the iris_dataset
#All three classes have a length of train dataset n=50. The first method, has k=5 parameters (four different means and one 
#different parameter in the Covariance Matrix). The second method has k=20 parameters (four different means and 16 different
#parametrs in the covariance matrix) The third method has k=8 parameters (4 different means and four differnet parameters - four 
#different variances of each gaussian distibution). The fourth method is non parametric with k=1, only the width of the parzen
#window. So for the first three methods, since n/k<40, AICc method will be used, and for the fourth AIC will be used.

setosa_AIC={}
versicolor_AIC={}
virginica_AIC={}
setosa_BIC={}
versicolor_BIC={}
virginica_BIC={}
n=50

#2a)
#setosa
Π=1
k=5
for count in range(len(setosa)):
    Π=Π*setosa_pdf_1(setosa[count])
setosa_AIC['2a']=-2*math.log(Π)+2*k+(2*k+1)/(n-k-1)
setosa_BIC['2a']=-2*math.log(Π)+k*math.log(n)

#versicolor
Π=1
k=5
for count in range(len(versicolor)):
    Π=Π*versicolor_pdf_1(versicolor[count])
versicolor_AIC['2a']=-2*math.log(Π)+2*k+(2*k+1)/(n-k-1)
versicolor_BIC['2a']=-2*math.log(Π)+k*math.log(n)

#virginica
Π=1
k=5
for count in range(len(virginica)):
    Π=Π*virginica_pdf_1(virginica[count])
virginica_AIC['2a']=-2*math.log(Π)+2*k+(2*k+1)/(n-k-1)
virginica_BIC['2a']=-2*math.log(Π)+k*math.log(n)

#2b)
#setosa
Π=1
k=20
for count in range(len(setosa)):
    Π=Π*setosa_pdf_2(setosa[count])
setosa_AIC['2b']=-2*math.log(Π)+2*k+(2*k+1)/(n-k-1)
setosa_BIC['2b']=-2*math.log(Π)+k*math.log(n)

#versicolor
Π=1
k=20
for count in range(len(versicolor)):
    Π=Π*versicolor_pdf_2(versicolor[count])
versicolor_AIC['2b']=-2*math.log(Π)+2*k+(2*k+1)/(n-k-1)
versicolor_BIC['2b']=-2*math.log(Π)+k*math.log(n)

#virginica
Π=1
k=20
for count in range(len(virginica)):
    Π=Π*virginica_pdf_2(virginica[count])
virginica_AIC['2b']=-2*math.log(Π)+2*k+(2*k+1)/(n-k-1)
virginica_BIC['2b']=-2*math.log(Π)+k*math.log(n)


#2c)
#setosa
Π=1
k=8
for count in range(len(setosa)):
    Π=Π*setosa_pdf_3(setosa[count])
setosa_AIC['2c']=-2*math.log(Π)+2*k+(2*k+1)/(n-k-1)
setosa_BIC['2c']=-2*math.log(Π)+k*math.log(n)

#versicolor
Π=1
k=8
for count in range(len(versicolor)):
    Π=Π*versicolor_pdf_3(versicolor[count])
versicolor_AIC['2c']=-2*math.log(Π)+2*k+(2*k+1)/(n-k-1)
versicolor_BIC['2c']=-2*math.log(Π)+k*math.log(n)

#virginica
Π=1
k=8
for count in range(len(virginica)):
    Π=Π*virginica_pdf_3(virginica[count])
virginica_AIC['2c']=-2*math.log(Π)+2*k+(2*k+1)/(n-k-1)
virginica_BIC['2c']=-2*math.log(Π)+k*math.log(n)

#2d)
#setosa
Π=1
k=1
for count in range(len(setosa)):
    Π=Π*setosa_pdf_4(setosa[count])
setosa_AIC['2d']=-2*math.log(Π)+2*k
setosa_BIC['2d']=-2*math.log(Π)+k*math.log(n)

#versicolor
Π=1
k=8
for count in range(len(versicolor)):
    Π=Π*versicolor_pdf_4(versicolor[count])
versicolor_AIC['2d']=-2*math.log(Π)+2*k
versicolor_BIC['2d']=-2*math.log(Π)+k*math.log(n)

#virginica
Π=1
k=8
for count in range(len(virginica)):
    Π=Π*virginica_pdf_4(virginica[count])
virginica_AIC['2d']=-2*math.log(Π)+2*k
virginica_BIC['2d']=-2*math.log(Π)+k*math.log(n)


In [ ]:
# Then for the pima dataset
#The two classes (yes_diabetes - no_diabetes) have a length of train dataset n1=268 and n2=500 respectively. The first method, 
#has k=9 parameters(8 different means and one different parameter in the Covariance Matrix). The second method has k=72 
#parameters (8 different means and 64 different parameters in the covariance matrix) 
#The third method has k=16 parameters (8 different means 8 four differnet parameters - 8 different variances of each 
#gaussian distibution). The fourth method is non parametric with k=1, only the width of the parzen
#window. So for the first three methods (except method 2a for no_diabetes), since n/k<40, AICc method will be used, 
#and for the fourth AIC will be used.

yes_diabetes_AIC={}
no_diabetes_AIC={}
yes_diabetes_BIC={}
no_diabetes_BIC={}
n1=268
n2=500

#2a)
k=9

#no_diabetes
Π=0
for count in range(len(no_diabetes)):
    Π+=math.log(no_diabetes_pdf_1(no_diabetes[count]))
no_diabetes_AIC['2a']=-2*Π+2*k
no_diabetes_BIC['2a']=-2*Π+k*math.log(n2)

#yes_diabetes
Π=0
for count in range(len(yes_diabetes)):
     Π+=math.log(yes_diabetes_pdf_1(yes_diabetes[count]))
yes_diabetes_AIC['2a']=-2*Π+2*k+(2*k+1)/(n1-k-1)
yes_diabetes_BIC['2a']=-2*Π+k*math.log(n1)


#2b)
k=72

#no_diabetes
Π=0
for count in range(len(no_diabetes)):
    Π+=math.log(no_diabetes_pdf_2(no_diabetes[count]))
no_diabetes_AIC['2b']=-2*Π+2*k+(2*k+1)/(n1-k-1)
no_diabetes_BIC['2b']=-2*Π+k*math.log(n2)

#yes_diabetes
Π=0
for count in range(len(yes_diabetes)):
     Π+=math.log(yes_diabetes_pdf_2(yes_diabetes[count]))
yes_diabetes_AIC['2b']=-2*Π+2*k+(2*k+1)/(n1-k-1)
yes_diabetes_BIC['2b']=-2*Π+k*math.log(n1)


#2c)
k=16

#no_diabetes
Π=0
for count in range(len(no_diabetes)):
    Π+=math.log(no_diabetes_pdf_3(no_diabetes[count]))
no_diabetes_AIC['2c']=-2*Π+2*k+(2*k+1)/(n1-k-1)
no_diabetes_BIC['2c']=-2*Π+k*math.log(n2)

#yes_diabetes
Π=0
for count in range(len(yes_diabetes)):
     Π+=math.log(yes_diabetes_pdf_3(yes_diabetes[count]))
yes_diabetes_AIC['2c']=-2*Π+2*k+(2*k+1)/(n1-k-1)
yes_diabetes_BIC['2c']=-2*Π+k*math.log(n1)


#2d)
k=1

#no_diabetes
Π=0
for count in range(len(no_diabetes)):
    Π+=math.log(no_diabetes_pdf_4(no_diabetes[count]))
no_diabetes_AIC['2d']=-2*Π+2*k
no_diabetes_BIC['2d']=-2*Π+k*math.log(n2)

#yes_diabetes
Π=0
for count in range(len(yes_diabetes)):
     Π+=math.log(yes_diabetes_pdf_4(yes_diabetes[count]))
yes_diabetes_AIC['2d']=-2*Π+2*k
yes_diabetes_BIC['2d']=-2*Π+k*math.log(n1)


In [ ]:
print ('The AIC score for Class No_DiAbetes is: ',no_diabetes_AIC,'\n')
print ('The BIC score for Class No_DiAbetes is: ',no_diabetes_BIC,'\n')
print ('The AIC score for Class Yes_DiAbetes is: ',yes_diabetes_AIC,'\n')
print ('The BIC score for Class Yes_DiAbetes is: ',yes_diabetes_BIC,'\n')
print ('The AIC score for Class setosa is: ',setosa_AIC,'\n')
print ('The BIC score for Class setosa is: ',setosa_BIC,'\n')
print ('The AIC score for Class virginica is: ' ,virginica_AIC,'\n')
print ('The BIC score for Class virginica is: ',virginica_BIC,'\n')
print ('The AIC score for Class versicolor is: ',versicolor_AIC,'\n')
print ('The BIC score for Class versicolor is: ',versicolor_BIC,'\n')


The AIC score for Class No_DiAbetes is:  {'2a': 42855.42917658481, '2b': 27258.012228253254, '2c': 27853.19067491427, '2d': 26126.26832363787} 

The BIC score for Class No_DiAbetes is:  {'2a': 42893.36064947061, '2b': 27560.720421596063, '2c': 27920.492930385437, '2d': 26130.482931736293} 

The AIC score for Class Yes_DiAbetes is:  {'2a': 24480.971847085468, '2b': 15373.93813892561, '2c': 15507.527366447659, '2d': 14259.57978071183} 

The BIC score for Class Yes_DiAbetes is:  {'2a': 24513.217086499215, '2b': 15631.745611778802, '2c': 15564.851684032246, '2d': 14263.170767692342} 

The AIC score for Class setosa is:  {'2a': 94.94668240182668, '2b': -46.79638879069642, '2c': -19.26144711916602, '2d': 522.7815751781123} 

The BIC score for Class setosa is:  {'2a': 104.2567974289674, '2b': -9.969721785581783, '2c': -4.379897222082317, '2d': 524.6935981835405} 

The AIC score for Class virginica is:  {'2a': 302.53967708918515, '2b': 158.59574100887042, '2c': 232.8430904010636, '2d': 561.502

So as shown above, both AIC and BIC has concluded that the best models for predicting each class of iris data are ranked as following:

1st) Model 2b

2nd) Model 2c

3rd) Model 2a

4th) Model 2d


For pima data, the best models for predicting each class of iris data are ranked as follwing:

1st) Model 2d

2nd) Model 2b

3rd) Model 2c

4th) Model 2a




### Task 3

In [ ]:
#Firstly for the iris_data
#Since from the initial iris_data each class appears 50 times, we conclude that the 3 classes are equi- probable. Therefore, 
#each time the classification of each datapoint of the test-data array will be performed only by calculating the greatest pdf
#respectively. Α 10-fold cross-validation will be implemented.

col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Class']
iris_data = pd.read_csv('UCIdata-exercise1/iris.data', names = col_names)

splited=partition(iris_data,10)
accuracy_list_3a=[]
accuracy_list_3b=[]
accuracy_list_3c=[]
accuracy_list_3d=[]


for i in range(len(splited)):
    splited_2=splited[:]
    test_data=splited_2.pop(i)
    train_data=pd.concat(splited_2)
    train_data.index=range(len(train_data))
    test_data.index=range(len(test_data))
    #Now we will create the three classes solely from the train_data of this iteration. From these classes each time
    #the pdf will be computed
    setosa=train_data[train_data["Class"] == 'Iris-setosa'].drop(["Class"], axis=1).to_numpy()
    versicolor=train_data[train_data["Class"] == 'Iris-versicolor'].drop(["Class"], axis=1).to_numpy()
    virginica=train_data[train_data["Class"] == 'Iris-virginica'].drop(["Class"], axis=1).to_numpy()
    
#############################################  
    #pdf for a) case
    # setosa Class
    Φ=setosa
    mean11=np.sum(Φ,axis=0)/setosa.shape[0]
    var=0
    for column in range(setosa.shape[1]):
        if np.var(setosa[:,column])>var:
            var=np.var(setosa[:,column])
    Σ11=np.identity(setosa.shape[1])*var    #calculate the covariaance matrix
    def setosa_pdf_3a (x): #x be a 4-dimensional vector
        pdf= multivariate_normal(mean11,Σ11).pdf(x)
        return pdf

    # versicolor Class
    Φ=versicolor
    mean21=np.sum(Φ,axis=0)/versicolor.shape[0] 
    var=0
    for column in range(versicolor.shape[1]):
        if np.var(versicolor[:,column])>var:
            var=np.var(versicolor[:,column])
    Σ21=np.identity(versicolor.shape[1])*var    #calculate the covariaance matrix
    def versicolor_pdf_3a (x): #x be a 4-dimensional vector
        pdf= multivariate_normal(mean21,Σ21).pdf(x)
        return pdf

    # virginica Class
    Φ=virginica
    mean31=np.sum(Φ,axis=0)/virginica.shape[0] 
    var=0
    for column in range(virginica.shape[1]):
        if np.var(virginica[:,column])>var:
            var=np.var(virginica[:,column])
    Σ31=np.identity(virginica.shape[1])*var    #calculate the covariaance matrix
    def virginica_pdf_3a (x): #x be a 4-dimensional vector
        pdf= multivariate_normal(mean31,Σ31).pdf(x)
        return pdf
    
###################################################3    
    #pdf for b) case
    # setosa
    Φ=setosa
    mean12=np.sum(Φ,axis=0)/setosa.shape[0] #case I.I.D dimensions
    sum_=0
    for i in range(setosa.shape[0]):
        sum_+=np.matmul((Φ[i]-mean12).reshape(4,1),np.transpose((Φ[i]-mean12).reshape(4,1)))
    Σ12=sum_/setosa.shape[0]
    def setosa_pdf_3b (x): #x be a 4-dimensional vector
        pdf= math.exp((-1/2)*np.matmul(np.matmul(np.transpose((x-mean12).reshape(4,1)),np.linalg.inv(Σ12)),(x-mean12).reshape(4,1)))/(((2*math.pi)**(setosa.shape[1]/2))*(np.linalg.det(Σ12))**(1/2))
        return pdf

    # versicolor
    Φ=versicolor
    mean22=np.sum(Φ,axis=0)/versicolor.shape[0] #case I.I.D dimensions
    sum_=0
    for i in range(versicolor.shape[0]):
        sum_+=np.matmul((Φ[i]-mean22).reshape(4,1),np.transpose((Φ[i]-mean22).reshape(4,1)))
    Σ22=sum_/versicolor.shape[0]
    def versicolor_pdf_3b (x): #x be a 4-dimensional vector
        pdf= math.exp((-1/2)*np.matmul(np.matmul(np.transpose((x-mean22).reshape(4,1)),np.linalg.inv(Σ22)),(x-mean22).reshape(4,1)))/(((2*math.pi)**(versicolor.shape[1]/2))*(np.linalg.det(Σ22))**(1/2))
        return pdf

    # virginica
    Φ=virginica
    mean32=np.sum(Φ,axis=0)/virginica.shape[0] #case I.I.D dimensions
    sum_=0
    for i in range(virginica.shape[0]):
        sum_+=np.matmul((Φ[i]-mean32).reshape(4,1),np.transpose((Φ[i]-mean32).reshape(4,1)))
    Σ32=sum_/virginica.shape[0]
    def virginica_pdf_3b (x): #x be a 4-dimensional vector
        pdf= math.exp((-1/2)*np.matmul(np.matmul(np.transpose((x-mean32).reshape(4,1)),np.linalg.inv(Σ32)),(x-mean32).reshape(4,1)))/(((2*math.pi)**(virginica.shape[1]/2))*(np.linalg.det(Σ32))**(1/2))
        return pdf

###################################################3    
    #pdf for c) case
    # setosa
    Φ=setosa
    mean13=(np.sum(Φ,axis=0)/setosa.shape[0]).tolist() #case I.I.D dimensions
    Σ13=[]
    for column in range(setosa.shape[1]):
        Σ13.append(np.var(setosa[:,column]))

    def setosa_pdf_3c (x): #x be a 4-dimensional vector
        pdf= pdf_naive(x,mean13,Σ13)
        return pdf

    #versicolor
    Φ=versicolor
    mean23=(np.sum(Φ,axis=0)/versicolor.shape[0]).tolist() #case I.I.D dimensions
    Σ23=[]
    for column in range(versicolor.shape[1]):
        Σ23.append(np.var(versicolor[:,column]))

    def versicolor_pdf_3c (x): #x be a 4-dimensional vector
        pdf= pdf_naive(x,mean23,Σ23)
        return pdf

    #virginica
    Φ=virginica
    mean33=(np.sum(Φ,axis=0)/virginica.shape[0]).tolist() #case I.I.D dimensions
    Σ33=[]
    for column in range(virginica.shape[1]):
        Σ33.append(np.var(virginica[:,column]))

    def virginica_pdf_3c (x): #x be a 4-dimensional vector
        pdf= pdf_naive(x,mean33,Σ33)
        return pdf
    
##################################################
    #pdf for d) case    
    h=3**(1/2)

    def setosa_pdf_3d(x):
        return kde_parzen_pdf_naive_bayes(setosa,x,h)
    
    def versicolor_pdf_3d(x):
        return kde_parzen_pdf_naive_bayes(versicolor,x,h)

    def virginica_pdf_3d(x):
        return kde_parzen_pdf_naive_bayes(virginica,x,h)   
    
###################################################
#starting the counting of correct predictions of each method for this iteration

    counter_accuracy_3a=0
    counter_accuracy_3b=0
    counter_accuracy_3c=0
    counter_accuracy_3d=0
    
    for row in range (len(test_data)):
        #method 3a
        pred1=setosa_pdf_3a(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred2=versicolor_pdf_3a(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred3=virginica_pdf_3a(test_data.drop(["Class"], axis=1).to_numpy()[row])
        if max(pred1,pred2,pred3)==pred1:
            if test_data.loc[row]['Class']=='Iris-setosa':
                counter_accuracy_3a+=1
        elif max(pred1,pred2,pred3)==pred2:
            if test_data.loc[row]['Class']=='Iris-versicolor':
                counter_accuracy_3a+=1      
        else:
            if test_data.loc[row]['Class']=='Iris-virginica':
                counter_accuracy_3a+=1    
        
        #method 3b
        pred1=setosa_pdf_3b(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred2=versicolor_pdf_3b(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred3=virginica_pdf_3b(test_data.drop(["Class"], axis=1).to_numpy()[row])
        if max(pred1,pred2,pred3)==pred1:
            if test_data.loc[row]['Class']=='Iris-setosa':
                counter_accuracy_3b+=1
        elif max(pred1,pred2,pred3)==pred2:
            if test_data.loc[row]['Class']=='Iris-versicolor':
                counter_accuracy_3b+=1      
        else:
            if test_data.loc[row]['Class']=='Iris-virginica':
                counter_accuracy_3b+=1   
        
        #method 3c
        pred1=setosa_pdf_3c(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred2=versicolor_pdf_3c(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred3=virginica_pdf_3c(test_data.drop(["Class"], axis=1).to_numpy()[row])
        if max(pred1,pred2,pred3)==pred1:
            if test_data.loc[row]['Class']=='Iris-setosa':
                counter_accuracy_3c+=1
        elif max(pred1,pred2,pred3)==pred2:
            if test_data.loc[row]['Class']=='Iris-versicolor':
                counter_accuracy_3c+=1      
        else:
            if test_data.loc[row]['Class']=='Iris-virginica':
                counter_accuracy_3c+=1
        
        #method 3d
        pred1=setosa_pdf_3d(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred2=versicolor_pdf_3d(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred3=virginica_pdf_3d(test_data.drop(["Class"], axis=1).to_numpy()[row])
        if max(pred1,pred2,pred3)==pred1:
            if test_data.loc[row]['Class']=='Iris-setosa':
                counter_accuracy_3d+=1
        elif max(pred1,pred2,pred3)==pred2:
            if test_data.loc[row]['Class']=='Iris-versicolor':
                counter_accuracy_3d+=1      
        else:
            if test_data.loc[row]['Class']=='Iris-virginica':
                counter_accuracy_3d+=1  
                  
    accuracy_list_3a.append(counter_accuracy_3a/len(test_data))
    accuracy_list_3b.append(counter_accuracy_3b/len(test_data))
    accuracy_list_3c.append(counter_accuracy_3c/len(test_data))
    accuracy_list_3d.append(counter_accuracy_3d/len(test_data))
    
final_accuracy_3a=mean(accuracy_list_3a)
final_accuracy_3b=mean(accuracy_list_3b)
final_accuracy_3c=mean(accuracy_list_3c)
final_accuracy_3d=mean(accuracy_list_3d)

print ('Accuracy of implementing the 3a algorithm and a 10-fold cross validation is: ',100*final_accuracy_3a,'%\n')
print ('Accuracy of implementing the 3b algorithm and a 10-fold cross validation is: ',100*final_accuracy_3b,'%\n')
print ('Accuracy of implementing the 3c algorithm and a 10-fold cross validation is: ',100*final_accuracy_3c,'%\n')
print ('Accuracy of implementing the 3d algorithm and a 10-fold cross validation is: ',100*final_accuracy_3d,'%\n')

Accuracy of implementing the 3a algorithm and a 10-fold cross validation is:  92.0 %

Accuracy of implementing the 3b algorithm and a 10-fold cross validation is:  98.0 %

Accuracy of implementing the 3c algorithm and a 10-fold cross validation is:  94.66666666666667 %

Accuracy of implementing the 3d algorithm and a 10-fold cross validation is:  92.0 %



From the 2nd answer, it was shown both from AIC and BIC, that the best methods were at order: Method 2b, 2c 2a and 2d. This is mostly verified by our 10-fold cross validation since we have achieved the same ranking, except that method 3a and 3d have 
achieved the same accuracy (92 %). The 3b method has achieved a result of 98 % accuracy, which is slighlty better than our best
NN classifier (8 NN as shown in exercise 1) which had averaged an accuracy of 96,44 % in 5-fold, 10-fold and 15-fold cross validation

In [ ]:
#Then for the pima_data
#For pima _data there are two seperate classes. From our original dataset there are 268 entries for class 1 (yes_diabetes)
#and 500 entries for class 0 (no_diabetes). We consider the assumption that P(yes_diabetes)=0.35 and P(no_diabetes)=0.65
#Therefore, each time the classification of each datapoint of the test-data array will be performed as following: 
#For each method, firstly, the pdf of each class for every entry of the dataset will be calculated.
#Then this entry will be classified to class=0 (no_diabetes), if pdf(x|class=0)*0,65>pdf(x|class=1)*0.35, or else the 
#opposite classification will be performed.

col_names = ['Pregnancies','Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age','Class']
pima_data = pd.read_csv('UCIdata-exercise1/pima-indians-diabetes.data', names = col_names)
for col in ['Glucose_Plasma','Blood Pressure','Skin THickness','Insulin','BMI','Diabetes Pedigree Function','Age']:
    val = pima_data[col].mean()
    pima_data[col] = pima_data[col].replace(0, val)

splited=partition(pima_data,10)
accuracy_list_3a=[]
accuracy_list_3b=[]
accuracy_list_3c=[]
accuracy_list_3d=[]

for i in range(len(splited)):
    splited_2=splited[:]
    test_data=splited_2.pop(i)
    train_data=pd.concat(splited_2)
    train_data.index=range(len(train_data))
    test_data.index=range(len(test_data))
    #Now we will create the two classes solely from the train_data of this iteration. From these classes each time
    #the pdf will be computed
    
    yes_diabetes=pima_data[pima_data["Class"] == 1].drop(["Class"], axis=1).to_numpy()
    no_diabetes=pima_data[pima_data["Class"] == 0].drop(["Class"], axis=1).to_numpy()
    
    #############################################  
    #pdf for a) case
    #yes_diabetes
    Φ=yes_diabetes
    mean41=np.sum(Φ,axis=0)/yes_diabetes.shape[0] #case I.I.D dimensions #case I.I.D dimensions
    var=0
    for column in range(yes_diabetes.shape[1]):
        if np.var(yes_diabetes[:,column])>var:
            var=np.var(yes_diabetes[:,column])
    Σ41=np.identity(yes_diabetes.shape[1])*var    #calculate the covariaance matrix
    def yes_diabetes_pdf_3a (x): #x be a 8-dimensional vector
        pdf= multivariate_normal(mean41,Σ41).pdf(x)
        return pdf

    #no_diabetes
    Φ=no_diabetes
    mean51=np.sum(Φ,axis=0)/no_diabetes.shape[0] 
    var=0
    for column in range(no_diabetes.shape[1]):
        if np.var(no_diabetes[:,column])>var:
            var=np.var(no_diabetes[:,column])
    Σ51=np.identity(no_diabetes.shape[1])*var    #calculate the covariaance matrix
    def no_diabetes_pdf_3a (x): #x be a 8-dimensional vector
        pdf= multivariate_normal(mean51,Σ51).pdf(x)
        return pdf
    
    ###################################################3    
    #pdf for b) case
    #yes_diabetes
    Φ=yes_diabetes
    mean42=np.sum(Φ,axis=0)/yes_diabetes.shape[0] #case I.I.D dimensions
    sum_=0
    for i in range(yes_diabetes.shape[0]):
        sum_+=np.matmul((Φ[i]-mean42).reshape(8,1),np.transpose((Φ[i]-mean42).reshape(8,1)))
    Σ42=sum_/yes_diabetes.shape[0]
    def yes_diabetes_pdf_3b (x): #x be a 8-dimensional vector
        pdf= math.exp((-1/2)*np.matmul(np.matmul(np.transpose((x-mean42).reshape(8,1)),np.linalg.inv(Σ42)),(x-mean42).reshape(8,1)))/(((2*math.pi)**(yes_diabetes.shape[1]/2))*(np.linalg.det(Σ42))**(1/2))
        return pdf

    #no_diabetes
    Φ=no_diabetes
    mean52=np.sum(Φ,axis=0)/no_diabetes.shape[0] #case I.I.D dimensions
    sum_=0
    for i in range(no_diabetes.shape[0]):
        sum_+=np.matmul((Φ[i]-mean52).reshape(8,1),np.transpose((Φ[i]-mean52).reshape(8,1)))
    Σ52=sum_/no_diabetes.shape[0]
    def no_diabetes_pdf_3b (x): #x be a 8-dimensional vector
        pdf= math.exp((-1/2)*np.matmul(np.matmul(np.transpose((x-mean52).reshape(8,1)),np.linalg.inv(Σ52)),(x-mean52).reshape(8,1)))/(((2*math.pi)**(no_diabetes.shape[1]/2))*(np.linalg.det(Σ52))**(1/2))
        return pdf
    
     ###################################################3    
    #pdf for c) case
    #yes_diabetes
    Φ=yes_diabetes
    mean43=(np.sum(Φ,axis=0)/yes_diabetes.shape[0]).tolist() #case I.I.D dimensions
    Σ43=[]
    for column in range(yes_diabetes.shape[1]):
        Σ43.append(np.var(yes_diabetes[:,column]))

    def yes_diabetes_pdf_3c (x): #x be a 4-dimensional vector
        pdf= pdf_naive(x,mean43,Σ43)
        return pdf

    #no_diabetes
    Φ=no_diabetes
    mean53=(np.sum(Φ,axis=0)/no_diabetes.shape[0]).tolist() #case I.I.D dimensions
    Σ53=[]
    for column in range(no_diabetes.shape[1]):
        Σ53.append(np.var(no_diabetes[:,column]))

    def no_diabetes_pdf_3c (x): #x be a 4-dimensional vector
        pdf= pdf_naive(x,mean53,Σ53)
        return pdf
    
    ###################################################3    
    #pdf for d) case
    #h will be the same for all 8 dimensions of all 2 differenct classes. We have 2 different patterns 
    #observed in the pima data (1,0) so the h will be the squareroot of 2
    h=2**(1/2)

    #yes_diabetes
    def yes_diabetes_pdf_3d(x):
        return kde_parzen_pdf_naive_bayes(yes_diabetes,x,h)

    #no_diabetes
    def no_diabetes_pdf_3d(x):
        return kde_parzen_pdf_naive_bayes(no_diabetes,x,h)
    
    ###################################################
#starting the counting of correct predictions of each method for this iteration

    counter_accuracy_3a=0
    counter_accuracy_3b=0
    counter_accuracy_3c=0
    counter_accuracy_3d=0
    
    for row in range (len(test_data)):
        #method 3a
        pred0=no_diabetes_pdf_3a(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred1=yes_diabetes_pdf_3a(test_data.drop(["Class"], axis=1).to_numpy()[row])
        if pred0*0.65>pred1*0.35:
            if test_data.loc[row]['Class']==0: #class 0 means no_diabetes - means chance 65 %
                counter_accuracy_3a+=1
        else:
            if test_data.loc[row]['Class']==1:
                counter_accuracy_3a+=1      
        
        #method 3b        
        pred0=no_diabetes_pdf_3b(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred1=yes_diabetes_pdf_3b(test_data.drop(["Class"], axis=1).to_numpy()[row])
        if pred0*0.65>pred1*0.35:
            if test_data.loc[row]['Class']==0: #class 0 means no_diabetes - means chance 65 %
                counter_accuracy_3b+=1
        else:
            if test_data.loc[row]['Class']==1:
                counter_accuracy_3b+=1      
        
        #method 3c 
        pred0=no_diabetes_pdf_3c(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred1=yes_diabetes_pdf_3c(test_data.drop(["Class"], axis=1).to_numpy()[row])
        if pred0*0.65>pred1*0.35:
            if test_data.loc[row]['Class']==0: #class 0 means no_diabetes - means chance 65 %
                counter_accuracy_3c+=1
        else:
            if test_data.loc[row]['Class']==1:
                counter_accuracy_3c+=1  
        
        #method 3d 
        pred0=no_diabetes_pdf_3d(test_data.drop(["Class"], axis=1).to_numpy()[row])
        pred1=yes_diabetes_pdf_3d(test_data.drop(["Class"], axis=1).to_numpy()[row])
        if pred0*0.65>pred1*0.35:
            if test_data.loc[row]['Class']==0: #class 0 means no_diabetes - means chance 65 %
                counter_accuracy_3d+=1
        else:
            if test_data.loc[row]['Class']==1:
                counter_accuracy_3d+=1  
                  
    accuracy_list_3a.append(counter_accuracy_3a/len(test_data))
    accuracy_list_3b.append(counter_accuracy_3b/len(test_data))
    accuracy_list_3c.append(counter_accuracy_3c/len(test_data))
    accuracy_list_3d.append(counter_accuracy_3d/len(test_data))
    
final_accuracy_3a=mean(accuracy_list_3a)
final_accuracy_3b=mean(accuracy_list_3b)
final_accuracy_3c=mean(accuracy_list_3c)
final_accuracy_3d=mean(accuracy_list_3d)

print ('Accuracy of implementing the 3a algorithm and a 10-fold cross validation is: ',100*final_accuracy_3a,'%\n')
print ('Accuracy of implementing the 3b algorithm and a 10-fold cross validation is: ',100*final_accuracy_3b,'%\n')
print ('Accuracy of implementing the 3c algorithm and a 10-fold cross validation is: ',100*final_accuracy_3c,'%\n')
print ('Accuracy of implementing the 3d algorithm and a 10-fold cross validation is: ',100*final_accuracy_3d,'%\n')

Accuracy of implementing the 3a algorithm and a 10-fold cross validation is:  65.63397129186603 %

Accuracy of implementing the 3b algorithm and a 10-fold cross validation is:  76.56869446343131 %

Accuracy of implementing the 3c algorithm and a 10-fold cross validation is:  74.87696514012305 %

Accuracy of implementing the 3d algorithm and a 10-fold cross validation is:  82.29835953520164 %



From the part of the 2nd exercise when AIC and BIC were calculated, it was shown  that the best methods were at order: Method 2d, 2b 2c and 2a. This is verified by our 10-fold cross validation since we have achieved the same ranking, as shown above. The 3d method has achieved a result of 82.29 % accuracy, which is  better than our bestNN classifier (17 NN as shown in exercise 1) which had averaged an accuracy of 73.82 % in 10-fold ,20 - fold and 30-fold cross validation

### Task 4

In [ ]:
# In order to examine whether the three classes of the iris_data are seperable by a hyperplane, we will perform the simple 
# perceptron algorithm. For this cause we will create the following function:

def step_func(z):
    return 1 if (z > 0) else 0

def perceptron(X, y, ε, epochs):
    
    # X are the Inputs.
    # y is labels/target (must consist of only 0s or 1s).
    # ε is learning rate.
    # epochs is the Number of iterations we want the algorithm to be executed
    
    # m is the number of training examples
    # n is the number of features 
    m, n = X.shape
    
    # Initializing parapeters(theta) to zeros.
    # +1 in n+1 for the bias term.
    theta = np.zeros((n+1,1))
    
    # Empty list to store how many examples were 
    # misclassified at every iteration.
    n_miss_list = []
    
    # Training.
    for epoch in range(epochs):
        
        # variable to store #misclassified.
        n_miss = 0
        
        # looping for every example.
        for idx, x_i in enumerate(X):
            
            # Insering 1 for bias, X0 = 1.
            x_i = np.insert(x_i, 0, 1).reshape(-1,1)
            
            # Calculating prediction/hypothesis.
            y_hat = step_func(np.dot(x_i.T, theta))
            
            # Updating if the example is misclassified.
            if (np.squeeze(y_hat) - y[idx]) != 0:
                theta =theta+ ε*((y[idx] - y_hat)*x_i)
                
                # Incrementing by 1.
                n_miss += 1
        
        # Appending number of misclassified examples
        # at every iteration.
        n_miss_list.append(n_miss)
        
    return theta, n_miss_list

In [ ]:
# There will be three examinations. For each examination, The X array will be the same. It will have length of 150, and 
#it will be compiled of 4 columns, each of them representing the dimensions that will be examined as inputs. Then, for each 
# run of the alforithm, three diferent y will be created. For example, for the examination of class setosa, y_setosa array 
# will be created in order to represent each setosa class as 1, and the other two classes as 0s.

X=iris_data.to_numpy()[:,:4]
y=iris_data.to_numpy()[:,4]
y_setosa=pd.get_dummies(y)['Iris-setosa'].to_numpy()
y_versicolor=pd.get_dummies(y)['Iris-versicolor'].to_numpy()
y_virginica=pd.get_dummies(y)['Iris-virginica'].to_numpy()


#Now we will run the algorithm for each class. We will implement a learning rate of 1 for all three runs,
#10 epochs for each class setosa and 400 epochs for the other two classes versicolor and virginica
theta_setosa, wrong_setosa=perceptron(X,y_setosa,1,10)

#versicolor
theta_versicolor, wrong_versicolor=perceptron(X,y_versicolor,1,400)

#virginica
theta_virginica, wrong_virginica=perceptron(X,y_virginica,1,400)

In [ ]:
print(wrong_setosa[-1])

0


As shown above, in only ten epochs the algorithm converged, and managed to seperate class setosa from the other two classes. This means that in fact, setosa is seperable by a hyperplane from the other two classes and we can find an equation of a hyperplane in 4-d space that can fully seperate setosa from the other two

In [ ]:
print(wrong_versicolor[-1])
print(wrong_virginica[-1])

6
4


On the other hand, both versicolor and virginica did not manage to converge, depsite the fact that the algorithm ran in 400 epochs each time. Indeed, since versicolor did not converge, and
 we showed that setosa converged, we expected that virginica would not converge as well. This is due to the fact that versicolor and virginica classes are mixed together and there is not a hyperplane in 4-d space that can seperate these two classes.

In [ ]:
print(min(wrong_versicolor))
print(min(wrong_virginica))

2
1


As shown above, the best thing that the simple perceptron could do in 400 epochs, was for versicolor to create a hyperplane that would missclassify only 2 datapoints, and for virginica to create a hyperplane in 4-d space that would misclassify 1 datapoint